# Задание 2

**Посчитать суммарную выручку за 2022 год по компаниям из отдельным компаниям из списка:**
* если у каких-то организаций нет выручки за 2022 год, то возьмите самую свежую доступную (за другой год);
* ответ округлите до целого.

## Описание датасетов

#### main.csv - *основная информация о предприятиях*
- ID - уникальный номер объекта;
- Name - название;
- Address - адрес

#### net_property.csv - *указание на сетевые предприятия*
- ID - уникальный номер объекта;
- IsNetObject - метка сетевого предприятия;

#### seats_property.csv - *количество посадочных мест*
- ID - уникальный номер объекта;
- SeatsCount - количество посадочных мест;
- ver_ID - номер версии записи

#### type_property.csv - *информация о специализации: столовая, закусочная, кафе, иное*
- ID - уникальный номер объекта;
- TypeObject - тип объекта;
- is_actual - актуальность записи

#### indicators_db.sqlite3 - *SQL-база с данными о выручке предприятий и числе сотрудников*
- данные за 2019 г.;
- данные за 2020-2022 гг.

#### Выручка_по_компаниям.xlsx - *XLSX-таблица со срезом ID требуемых компаний*

In [2]:
import pandas as pd
import numpy as np

### Чтение датасетов

In [3]:
df1 = pd.read_csv('main.csv')
df2 = pd.read_csv('net_property.csv')
df3 = pd.read_csv('seats_property.csv')
df4 = pd.read_csv('type_property.csv')

In [4]:
import sqlalchemy as sql

In [5]:
con = sql.create_engine('sqlite:///indicators_db.sqlite3')

In [6]:
df = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", con)

In [7]:
df

,type,name,tbl_name,rootpage,sql
0,table,org_indicators_2019,org_indicators_2019,2,CREATE TABLE org_indicators_2019 (\r\n\tID BIG...
1,table,org_indicators_2020_2022,org_indicators_2020_2022,490,CREATE TABLE org_indicators_2020_2022 (\r\n\tI...


In [8]:
df_sql_select_19 = pd.read_sql('select * from org_indicators_2019', con)

In [9]:
df_sql_select_20_22 = pd.read_sql('select * from org_indicators_2020_2022', con)

In [10]:
xlsx = pd.ExcelFile('Выручка_по_компаниям.xlsx')

In [11]:
xlsx.sheet_names

['ХитрыйПустойЛист', 'Лист1']

In [12]:
df_ex = pd.read_excel(xlsx, sheet_name = 'Лист1')

### Выборка необходимых колонок и разворот через pivot

In [13]:
df_19 = df_sql_select_19[df_sql_select_19['INDICATOR_NAME']  == 'Выручка']

In [14]:
df_19_piv = df_19.pivot(index = 'ID', columns = ['YEAR','INDICATOR_NAME'], values = 'VALUE')

In [15]:
df_20_22 = df_sql_select_20_22[df_sql_select_20_22['INDICATOR_NAME']  == 'Выручка']

In [16]:
df_20_22_piv = df_20_22.pivot(index = 'ID', columns = ['YEAR','INDICATOR_NAME'], values = 'VALUE')

In [17]:
df_19_piv

YEAR,2019
INDICATOR_NAME,Выручка
ID,
19630,20522186
19643,20577873
19677,34723333
19710,29753085
19721,32860968
...,...
375830,33223677
375848,36812489


In [18]:
df_20_22_piv

YEAR,2020,2021,2022
INDICATOR_NAME,Выручка,Выручка,Выручка
ID,,,
19630,31222228.0,21721921.0,NaN
19643,35670701.0,39125612.0,NaN
19647,34926614.0,23176004.0,27925131.0
19657,23410468.0,28065717.0,36074059.0
19677,NaN,30803884.0,25303592.0
...,...,...,...
375884,NaN,32408516.0,NaN
375933,36102422.0,21245735.0,39298414.0


### Удаление уровня "YEAR" и переименование колонок

In [110]:
testing_1 = df_19_piv.droplevel([0], axis = 1)
testing_1.columns.names = [None]
testing_1.to_csv('test_19.csv')
testing_1 = pd.read_csv('test_19.csv')
testing_1 = testing_1.rename(columns = {'Выручка':2019})

In [111]:
testing_2 = df_20_22_piv.droplevel([0], axis = 1)
testing_2.columns.names = [None]
testing_2.to_csv('test_20.csv')
testing_2 = pd.read_csv('test_20.csv')
rename = {'Выручка':2020, 'Выручка.1':2021, 'Выручка.2':2022}
testing_2 = testing_2.rename(columns = rename)

In [112]:
testing_1

,ID,2019
0,19630,20522186
1,19643,20577873
2,19677,34723333
3,19710,29753085
4,19721,32860968
...,...,...
6691,375830,33223677
6692,375848,36812489
6693,375954,21503857
6694,375956,25910529


In [113]:
testing = pd.merge(testing_1, testing_2, on = 'ID', how = 'outer')

In [114]:
df_ex = df_ex.drop_duplicates()

In [115]:
df_test_ex = pd.merge(testing, df_ex, on = 'ID', how='inner')

In [120]:
df_unstacked = df_test_ex.melt('ID').dropna()

In [130]:
sort = df_unstacked.sort_values(by = 'variable', ascending = True)

In [189]:
mask = sort.drop_duplicates(['ID'], keep = 'last')

In [190]:
mask

,ID,variable,value
4,206594,2019,22495241.0
28,198181,2020,23312117.0
25,357706,2020,33208127.0
23,287998,2020,25325896.0
18,27549,2020,22491442.0
33,24109,2021,20306338.0
62,264876,2022,37021556.0
48,20105,2022,31684347.0
51,201139,2022,32111693.0
53,229655,2022,23942018.0


In [191]:
mask['value'].sum()

462125109.0